<a href="https://colab.research.google.com/github/carolineoliveira994/100-days-of-code-python/blob/main/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets scikit-learn
!pip install python-dotenv
!pip install evaluate!pip install datasets
!pip install transformers torch
!pip install spacy
!python -m spacy download pt_core_news_sm

In [ ]:
!pip install evaluate
!pip install python-dotenv


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os
import evaluate
from dotenv import load_dotenv


print("Diretório atual:", os.getcwd())


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import evaluate
metric = evaluate.load('f1')

from sklearn.metrics import f1_score


In [ ]:
print("Arquivos no diretório atual:", os.listdir())


In [ ]:
with open('.env', 'w') as f:
    f.write("""
BLUESKY_APP_USER = 'cocorolini.bsky.social'
BLUESKY_APP_PASS='3ovj-gbnh-trwd-k66r'

""")

In [ ]:
!python main.py search BOULOS --sort latest --limit 90


In [ ]:
from google.colab import files
import pandas as pd
# Defina o caminho correto para o arquivo CSV
file_path = './data/search_results_BOULOS_2024_09_19.csv'
df = pd.read_csv(file_path)

df.head()


In [ ]:
import spacy

# Carregar o modelo de língua portuguesa
nlp = spacy.load('pt_core_news_sm')

# Função de pré-processamento usando spaCy
def preprocess_text_spacy(text_at):
    # Aplicar o modelo spaCy ao texto
    doc = nlp(text_at)
    # Extrair palavras lematizadas e remover stopwords e pontuação
    words = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(words)

# Aplicar pré-processamento ao DataFrame
df['text_at'] = df['text'].apply(preprocess_text_spacy)  # Substitua 'post_content' pelo nome real da coluna


In [ ]:
print(df[['text', 'text_at']].head())


In [ ]:
df = pd.read_csv('./data/search_results_BOULOS_2024_09_19.csv')


In [ ]:
df

In [ ]:
pd.set_option('display.max_columns', None)
print(df)

In [ ]:
def clean_text(text):
    # Remover URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remover menções (@username)
    text = re.sub(r'@\w+', '', text)

    # Remover hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # Remover números
    text = re.sub(r'\d+', '', text)

    # Remover pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)

    # Remover múltiplos espaços em branco
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [ ]:
# Carregar o tokenizer para o modelo BERT em português
tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")


In [ ]:
columns_to_remove = ["reply_count", "author_display_name", "author_handle", "indexed_at", "cid", "uri", "created_at", "repost_count", "like_count"]
df = df.drop(columns=columns_to_remove)

# Exibir DataFrame após remover colunas
print("\nDepois:")
print(df)


In [ ]:
import spacy

# Carregar o modelo em português
nlp = spacy.load("pt_core_news_sm")

# Função para pré-processamento de texto
def preprocess_text_spacy(text_at):
    # Aplicar o modelo spaCy ao texto
    doc = nlp(text_at)
    # Extrair palavras lematizadas e remover stopwords e pontuação
    words = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(words)

# Aplicar o pré-processamento ao DataFrame
df['text_at'] = df['text'].apply(preprocess_text_spacy)


In [ ]:
df

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download necessário para o VADER
nltk.download('vader_lexicon')

# Criar um analisador de sentimento
sia = SentimentIntensityAnalyzer()

# Assumindo que 'df' é seu DataFrame e 'text-at' é a coluna de texto
def calcular_sentimento(texto):
    score = sia.polarity_scores(texto)['compound']
    return 'positivo' if score > 0 else 'negativo'

# Aplicar a função a cada linha da coluna 'text-at' do DataFrame
df['label'] = df['text_at'].apply(calcular_sentimento)

print(df)

In [ ]:
label_mapping = {
    'positivo': 1,
    'negativo': 0
}

# Aplicar a conversão
df['label'] = df['label'].map(label_mapping)


In [ ]:
print(df['label'].unique())  # Deve mostrar apenas números, como [0, 1]


In [ ]:
from transformers import BertTokenizer

# Carregar o tokenizador do BERT em português
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Tokenizar os textos já pré-processados
def tokenize_text(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)

# Aplicar a tokenização aos textos pré-processados
df['tokenized'] = df['text_at'].apply(tokenize_text)

# Exibir um exemplo tokenizado
print(df['tokenized'].iloc[0])


In [ ]:
import pandas as pd
from datasets import Dataset

# Criar novas colunas no DataFrame extraindo as informações do tokenized
df['input_ids'] = df['tokenized'].apply(lambda x: x['input_ids'])
df['attention_mask'] = df['tokenized'].apply(lambda x: x['attention_mask'])

# Agora o DataFrame tem colunas de listas simples, que podem ser convertidas para um Dataset
df_simplificado = df[['input_ids', 'attention_mask', 'label']]  # 'label' é sua coluna de rótulos

# Converter para o Dataset do Hugging Face
dataset = Dataset.from_pandas(df_simplificado)


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Carregar o modelo BERT pré-treinado para classificação de sentimentos
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=2)  # 'num_labels' depende do número de classes (ex: 2 para positivo/negativo)

# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./results',           # Diretório para salvar os resultados
    num_train_epochs=3,               # Número de épocas
    per_device_train_batch_size=8,    # Tamanho do batch por dispositivo durante o treinamento
    per_device_eval_batch_size=16,    # Tamanho do batch por dispositivo durante a avaliação
    evaluation_strategy="epoch",      # Avaliação a cada época
    save_total_limit=2,               # Limitar o número de checkpoints salvos
)

# Definir o treinador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,            # Dataset de treinamento
    eval_dataset=dataset,             # Dataset de avaliação
)

# Iniciar o treinamento
trainer.train()


In [ ]:
# Avaliar o modelo
eval_results = trainer.evaluate()

# Exibir resultados
print(f"Resultados da avaliação: {eval_results}")


In [ ]:
# Fazer previsões
predictions = trainer.predict(test_dataset)

# Exibir as predições
print(predictions)


In [ ]:
from sklearn.metrics import classification_report

# Obtenha as predições e os rótulos verdadeiros
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Calcular o relatório de classificação
report = classification_report(labels, preds, target_names=['negativo', 'positivo'])
print(report)



In [ ]:
!python main.py search GUILHERMEBOULOS --sort latest --limit 90


In [ ]:
from google.colab import files
import pandas as pd
# Defina o caminho correto para o arquivo CSV
file_path = './data/search_results_GUILHERMEBOULOS_2024_09_19.csv'
df_test = pd.read_csv(file_path)

df_test.head()

In [ ]:
def label_text(text):
    positive_keywords = ['ótimo', 'bom', 'maravilhoso', 'adoro', 'excelente', 'Prefeito', 'incrível', 'demais', 'casa comigo', 'delicia', 'Boulos50', 'boulos50', 'vitória', 'Vitória', 'PREFEITO', 'VITÓRIA', 'VITORIA', 'proposta', 'conhece']
    negative_keywords = ['ruim', 'horrível', 'pior', 'não recomendo']

    text = text.lower()

    if any(word in text for word in positive_keywords):
        return 1  # Positivo
    elif any(word in text for word in negative_keywords):
        return 0  # Negativo
    else:
        return -1  # Neutro ou indefinido

# Aplicar a função para rotular automaticamente os textos
df_test['label'] = df_test['text'].apply(label_text)

# Exibir o DataFrame com os labels adicionados
print(df_test)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer
import numpy as np

# Carregar o tokenizador e o modelo
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Função para rotular automaticamente os textos
def label_with_model(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
    outputs = model(**inputs)
    predictions = np.argmax(outputs.logits.detach().numpy(), axis=1)
    return predictions

# Aplicar a função de rotulagem automática para todos os textos
df_test['label'] = label_with_model(df_test['text'].tolist())

# Exibir o DataFrame com os labels gerados automaticamente
print(df_test)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer
import numpy as np

# Carregar o tokenizador e o modelo
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Função para rotular automaticamente os textos
def label_with_model(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
    outputs = model(**inputs)
    predictions = np.argmax(outputs.logits.detach().numpy(), axis=1)
    return predictions

# Aplicar a função de rotulagem automática para todos os textos
df_test['label'] = label_with_model(df_test['text'].tolist())

# Exibir o DataFrame com os labels gerados automaticamente
print(df_test)


In [ ]:
df_test

In [ ]:
from sklearn.model_selection import train_test_split

# Separar o conjunto de dados em treino (80%) e teste (20%)
train_df, test_df = train_test_split(df_test, test_size=0.2, random_state=42)

# Exibir as primeiras linhas dos dados de treino e teste
print(train_df.head())
print(test_df.head())


In [ ]:
from transformers import BertTokenizer

# Inicializar o tokenizador
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Tokenizar os textos de treinamento e teste
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True, max_length=128)


In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Criar datasets de treino e teste
train_dataset = CustomDataset(train_encodings, train_df['label'].tolist())
test_dataset = CustomDataset(test_encodings, test_df['label'].tolist())


In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Criar datasets de treino e teste
train_dataset = CustomDataset(train_encodings, train_df['label'].tolist())
test_dataset = CustomDataset(test_encodings, test_df['label'].tolist())


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Carregar o modelo BERT para classificação
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=2)

# Definir os argumentos do treinamento
training_args = TrainingArguments(
    output_dir='./results',          # Pasta para salvar os resultados
    evaluation_strategy="epoch",     # Avaliar ao final de cada época
    per_device_train_batch_size=16,  # Tamanho do lote
    per_device_eval_batch_size=16,   # Tamanho do lote para avaliação
    num_train_epochs=3,              # Número de épocas
    weight_decay=0.01,               # Taxa de decaimento
)

# Inicializar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Iniciar o treinamento
trainer.train()


In [ ]:
# Avaliar o modelo no conjunto de teste
eval_results = trainer.evaluate()

# Exibir os resultados da avaliação
print(eval_results)


In [ ]:
# Fazer previsões no conjunto de teste
predictions = trainer.predict(test_dataset)

# Exibir as previsões
print(predictions.predictions)


In [ ]:
model.save_pretrained('./modelo_final')
tokenizer.save_pretrained('./modelo_final')


In [ ]:
model.save_pretrained('./meu_modelo')
tokenizer.save_pretrained('./meu_modelo')
